# 8. 树

## 8.1 树的嵌套列表实现
用递归的嵌套列表实现一个二叉树。

```[root, left, right]```: root表示根节点（值），left和right分别表示左右子树，也是一个列表。

In [1]:
myTree = ['a',  # 树根
          ['b', # 左子树
           ['d', [], []],
           ['e', [], []]],
          ['c', # 右子树
           ['f', [], []],
           ['g', [], []]]]

In [2]:
def BinaryTree(r):  # 创建一个二叉树 只有根节点
    return [r, [], []]

def insertLeft(root, newBranch):  # 在左子树插入新节点
    t = root.pop(1)  # 移除索引1的元素（左子树），同时返回这个元素
    if len(t) > 1:  # 判断原左子树是否为空
        root.insert(1, [newBranch, t, []])  # 新节点插入到索引1的位置，原左子树作为新节点的左子树
    else:
        root.insert(1, [newBranch, [], []])  # 新节点作为左子树 ，原左子树为空  
    return root

def insertRight(root, newBranch):  # 在右子树插入新节点
    t = root.pop(2)  # 弹出右子树
    if len(t) > 1:  # 如果右子树不为空
        root.insert(2, [newBranch, [], t])  # 新节点作为右子树，原右子树作为新节点的右子树
    else:
        root.insert(2, [newBranch, [], []])  # 新节点作为右子树，原右子树为空
    return root

def getRootVal(root):  # 获取根节点的值
    return root[0]  # 根节点的值在索引0处

def setRootVal(root, newVal):  # 设置根节点的值
    root[0] = newVal    # 将新值赋给索引0处

def getLeftChild(root):  # 获取左子树
    return root[1]  # 左子树在索引1处

def getRightChild(root):  # 获取右子树
    return root[2]

r = BinaryTree(3)   # 创建一个根节点为3的二叉树
insertLeft(r, 4)    # 在左子树插入4
insertLeft(r, 5)    # 在左子树插入5
insertRight(r, 6)   # 在右子树插入6
insertRight(r, 7)   # 在右子树插入7
print(r)  # [3, [5, [4, [], []], []], [7, [], [6, [], []]]]
print(getRootVal(r))  # 3
setRootVal(r, 9)
print(getRootVal(r))  # 9
print(getLeftChild(r))  # [5, [4, [], []], []]
print(getRightChild(r))  # [7, [], [6, [], []]]

[3, [5, [4, [], []], []], [7, [], [6, [], []]]]
3
9
[5, [4, [], []], []]
[7, [], [6, [], []]]


## 8.2 树的链表实现
用节点链接法来实现树，每个节点保存根节点的数据项，以及指向左右子树的链接。

In [3]:
class BinaryTree:
    def __init__(self, rootObj):
        self.key = rootObj  # 根节点数据项
        self.leftChild = None   
        self.rightChild = None

    def insertLeft(self, newNode):
        if self.leftChild is None:  # 如果左子树为空
            self.leftChild = BinaryTree(newNode)  # 直接插入新节点
        else:   # 左子树不为空 类似于链表插入
            t = BinaryTree(newNode)  # 创建新节点
            t.leftChild = self.leftChild  # 将原左子树作为新节点的左子树
            self.leftChild = t  # 新节点作为当前节点的左子树
    
    def insertRight(self, newNode):
        if self.rightChild is None:  # 如果右子树为空
            self.rightChild = BinaryTree(newNode)  # 直接插入新节点
        else:
            t = BinaryTree(newNode)  # 创建新节点
            t.rightChild = self.rightChild  # 将原右子树作为新节点的右子树
            self.rightChild = t  # 新节点作为当前节点的右子树

    def getRightChild(self):
        return self.rightChild  # 获取右子树
    def getLeftChild(self):
        return self.leftChild   # 获取左子树
    def setRootVal(self, obj):
        self.key = obj  # 设置根节点的值
    def getRootVal(self):
        return self.key # 获取根节点的值

r = BinaryTree(3)   # 创建一个根节点为3的二叉树
r.insertLeft(4)    # 在左子树插入4
r.insertLeft(5)    # 在左子树插入5
r.insertRight(6)   # 在右子树插入6
r.insertRight(7)   # 在右子树插入7
print(r.getRootVal())  # 3
print(r.getLeftChild().getRootVal())  # 5
print(r.getRightChild().getRootVal())  # 7
print(r.getLeftChild().getLeftChild().getRootVal())  # 4
print(r.getRightChild().getRightChild().getRootVal())  # 6

3
5
7
4
6


## 8.3 表达式解析
1. 从全括号表达式构建树；
2. 利用表达式解析树求值；
3. 从表达式解析树恢复原始的字符串表达式；

步骤：
1. 全括号表达式分解为token：分为括号、操作符、操作数几类；
2. 读取：初始化当前节点（根节点），读取左括号则当前节点下降，右括号则上升；读取到操作符，则设置当前节点值为操作符，并新建右节点；读取操作数，则将当前节点值设为操作数，当前节点上升到父节点。

思路：
1. 创建左右子树用insertLeft/Right；
2. 当前节点设置用setRootVal；
3. 下降用getLeft/Right；
4. 上升节点用一个栈来记录跟踪父节点：节点下降时，将下降前的节点push入栈；需要上升时直接pop出栈的节点即可。

In [4]:
# step1 创建表达式解析树
def buildParseTree(fpexp):
    fplist = fpexp.split()  # 将表达式字符串拆分为列表
    pStack = []  # 创建一个栈用于存储父节点
    eTree = BinaryTree('')  # 创建一个空的二叉树
    pStack.append(eTree)  # 将树压入栈中
    currentTree = eTree  # 当前树指向eTree

    for i in fplist:
        if i == '(':  # 遇到左括号，创建左子树
            currentTree.insertLeft('')  # 插入一个空节点作为左子树
            pStack.append(currentTree)  # 将当前树压入栈中
            currentTree = currentTree.getLeftChild()  # 移动到左子树

        elif i not in ['+', '-', '*', '/', ')']:  # 遇到操作数
            currentTree.setRootVal(int(i))  # 设置当前节点的值为操作数
            parent = pStack.pop()  # 弹出栈顶元素作为父节点
            currentTree = parent  # 当前树指向父节点 出栈上升
    
        elif i in ['+', '-', '*', '/']:  # 遇到运算符
            currentTree.setRootVal(i)  # 设置当前节点的值为运算符
            currentTree.insertRight('')  # 插入一个空节点作为右子树
            pStack.append(currentTree)  # 将当前树压入栈中
            currentTree = currentTree.getRightChild()  # 移动到右子树

        elif i == ')':  # 遇到右括号，完成当前子树的构建
            currentTree = pStack.pop()  # 弹出栈顶元素作为当前树 出栈上升
        else:
            raise ValueError("Unknown Operator: " + i)
    return eTree
pt = buildParseTree("( ( 10 + 5 ) * 3 )")
print(pt.getRootVal())  # *
print(pt.getLeftChild().getRootVal())  # +
print(pt.getLeftChild().getLeftChild().getRootVal())  # 10
print(pt.getLeftChild().getRightChild().getRootVal())  # 5
print(pt.getRightChild().getRootVal())  # 3

*
+
10
5
3


In [5]:
# step2 计算表达式解析树的值
import operator
def evaluate(parseTree):
    opers = {'+': operator.add, '-': operator.sub,
             '*': operator.mul, '/': operator.truediv}  # 定义运算符对应的函数
    leftC = parseTree.getLeftChild()  # 获取左子树
    rightC = parseTree.getRightChild()  # 获取右子树

    if leftC and rightC:  # 如果有左子树和右子树
        fn = opers[parseTree.getRootVal()]  # 获取当前节点的运算符对应的函数
        return fn(evaluate(leftC), evaluate(rightC))  # 递归计算左子树和右子树的值，并应用运算符函数
    else:
        return parseTree.getRootVal()  # 如果是叶节点，返回节点的值
print(evaluate(pt))  # 45

45


## 8.4 树的遍历
1. 前序遍历：根 -> 左子树 -> 右子树
2. 中序遍历：左子树 -> 根 -> 右子树
3. 后序遍历：左子树 -> 右子树 -> 根

In [6]:
def preorder(tree):
    '''前序遍历'''
    if tree:
        print(tree.getRootVal())
        preorder(tree.getLeftChild())
        preorder(tree.getRightChild())

In [7]:
def postorder(tree):
    '''后序遍历'''
    if tree != None:
        postorder(tree.getLeftChild())
        postorder(tree.getRightChild())
        print(tree.getRootVal())

In [8]:
def inorder(tree):
    '''中序遍历'''
    if tree != None:
        inorder(tree.getLeftChild())
        print(tree.getRootVal())
        inorder(tree.getRightChild())

In [9]:
# 后序遍历：表达式求值
def postordereval(tree):
    opers = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}
    res1 = None
    res2 = None
    if tree:
        res1 = postordereval(tree.getLeftChild())
        res2 = postordereval(tree.getRightChild())

        if res1 and res2:
            return opers[tree.getRootVal()](res1, res2) # 查询根节点操作符 随后进行计算
        else:
            return tree.getRootVal()

In [10]:
# 生成全括号中缀表达式
def printexp(tree):
    sVal = ""
    if tree:
        sVal = '(' + printexp(tree.getLeftChild())
        sVal = sVal + str(tree.getRootVal())
        sVal = sVal + printexp(tree.getRightChild())
    return sVal

## 8.5 二叉堆
优先队列（priority queue）：高优先级排在队首，低优先级排后面，增加了入队的复杂度。可以使用二叉堆实现优先队列，让入队和出队复杂度保持在对数水平。

二叉堆本质上是一种完全二叉树（Complete Binary Tree），它通常用于实现优先队列（Priority Queue）。 

1. 结构性质 (Structure Property)
二叉堆必须是一棵完全二叉树。这意味着：

紧凑排列：除了最后一层外，其他所有层的节点都是满的。

左对齐：最后一层的节点必须从左到右依次排列，中间不能有空缺。 

这一性质使得二叉堆非常适合用数组来存储，而不需要指针，极大地节省了空间。

2. 堆序性质 (Heap Order Property)
根据节点值的排列规则，二叉堆分为两种：

最大堆 (Max-Heap)：父节点的值 $\ge$ 任意子节点的值。 

根节点是整个堆中的最大值。

最小堆 (Min-Heap)：父节点的值 $\le$ 任意子节点的值。 

根节点是整个堆中的最小值。

3. 数组存储与索引关系
由于是完全二叉树，二叉堆通常使用数组实现。假设根节点索引为 0，对于任意节点 i：

父节点 (Parent): $\lfloor \frac{i-1}{2} \rfloor$

左子节点 (Left Child): $2i + 1$

右子节点 (Right Child): $2i + 2$

In [12]:
from pythonds.trees.binheap import BinHeap
bh = BinHeap()
bh.insert(5)
bh.insert(7)
bh.insert(3)
bh.insert(11)

print(bh.delMin())
print(bh.delMin())
print(bh.delMin())
print(bh.delMin())

3
5
7
11


In [13]:
class BinHeap:
    def __init__(self):
        self.heapList = [0]
        self.currentSize = 0


    def buildHeap(self,alist):  # 从无序表生成堆
        i = len(alist) // 2
        self.currentSize = len(alist)
        self.heapList = [0] + alist[:]
        print(len(self.heapList), i)
        while (i > 0):
            print(self.heapList, i)
            self.percDown(i)    # 逐个下沉
            i = i - 1
        print(self.heapList,i)
                        
    def percDown(self,i):
        while (i * 2) <= self.currentSize:
            mc = self.minChild(i)   # 最小的子节点
            if self.heapList[i] > self.heapList[mc]:    # 如果比最小的子节点大
                tmp = self.heapList[i]  # 交换
                self.heapList[i] = self.heapList[mc]
                self.heapList[mc] = tmp
            i = mc
                
    def minChild(self,i):
        if i * 2 + 1 > self.currentSize:
            return i * 2
        else:
            if self.heapList[i * 2] < self.heapList[i * 2 + 1]:
                return i * 2
            else:
                return i * 2 + 1

    def percUp(self,i):
        while i // 2 > 0:
            if self.heapList[i] < self.heapList[i//2]:  # 如果新节点比父节点小
               tmp = self.heapList[i // 2]  # 与父节点交换
               self.heapList[i // 2] = self.heapList[i]
               self.heapList[i] = tmp
            i = i // 2  # 沿路径向上
 
    def insert(self,k):
        self.heapList.append(k)
        self.currentSize = self.currentSize + 1
        # 沿着路径来上浮到其正确的位置 满足堆次序
        # 上浮不会影响其他路径的次序
        self.percUp(self.currentSize)

    def delMin(self):   # 移走堆中最小的key: 根节点heaplist[1]
        retval = self.heapList[1]
        # 为保持完全二叉树 只用最后一个节点来代替根节点
        self.heapList[1] = self.heapList[self.currentSize] 
        self.currentSize = self.currentSize - 1
        self.heapList.pop()
        self.percDown(1)    # 新的根节点沿着一条路径下沉 直到比两个子节点都小
        return retval
        
    def isEmpty(self):
        if self.currentSize == 0:
            return True
        else:
            return False


## 8.6 二叉查找树
比父节点小的key都在左子树，比父节点大的key都在右子树。首先插入的树成为树根，然后依次插入其他元素，插入顺序不同，BST也不同。

在类中使用特殊方法的作用，eg. ```__len__```
```python
# 使用普通方法
tree = BinarySearchTree()
print(tree.length())  # 需要显式调用方法

# 使用特殊方法后
print(len(tree))      # 更自然、更Pythonic
```

In [ ]:
class BinarySearchTree:
    def __init__(self):
        self.root = None
        self.size = 0
    def length(self):
        return self.size
    def __len__(self):  # 特殊方法：支持len(tree)
        return self.size
    def __iter__(self):
        return self.root.__iter__()    # 特殊方法：支持迭代

In [15]:
class TreeNode:
    def __init__(self, key, val, left=None, right=None, parent=None):
        self.key = key
        self.payload = val  # 存储的值
        self.leftChild = left
        self.rightChild = right
        self.parent = parent
    
    def hasLeftChild(self):
        """检查是否有左子节点"""
        return self.leftChild is not None
    
    def hasRightChild(self):
        """检查是否有右子节点"""
        return self.rightChild is not None
    
    def isLeftChild(self):
        """检查是否是父节点的左子节点"""
        return self.parent and self.parent.leftChild == self
    
    def isRightChild(self):
        """检查是否是父节点的右子节点"""
        return self.parent and self.parent.rightChild == self
    
    def isRoot(self):
        """检查是否是根节点"""
        return self.parent is None
    
    def isLeaf(self):
        """检查是否是叶子节点"""
        return not (self.hasLeftChild() or self.hasRightChild())
    
    def hasAnyChildren(self):
        """检查是否有任何子节点"""
        return self.hasLeftChild() or self.hasRightChild()
    
    def hasBothChildren(self):
        """检查是否同时有左右子节点"""
        return self.hasLeftChild() and self.hasRightChild()
    
    def replaceNodeData(self, key, value, lc, rc):
        """替换节点数据"""
        self.key = key
        self.payload = value
        self.leftChild = lc
        self.rightChild = rc
        if self.hasLeftChild():
            self.leftChild.parent = self
        if self.hasRightChild():
            self.rightChild.parent = self
    
    def findSuccessor(self):
        """找到后继节点"""
        succ = None
        if self.hasRightChild():
            # 后继节点是右子树中的最小节点
            succ = self.rightChild.findMin()
        else:
            if self.parent:
                if self.isLeftChild():
                    succ = self.parent
                else:
                    self.parent.rightChild = None
                    succ = self.parent.findSuccessor()
                    self.parent.rightChild = self
        return succ
    
    def findMin(self):
        """找到当前子树中的最小节点"""
        current = self
        while current.hasLeftChild():
            current = current.leftChild
        return current
    
    def __iter__(self):
        """中序遍历迭代器"""
        # 遍历左子树
        if self.hasLeftChild():
            for elem in self.leftChild:
                yield elem
        
        # 返回当前节点
        yield self
        
        # 遍历右子树
        if self.hasRightChild():
            for elem in self.rightChild:
                yield elem
    
    def __str__(self):
        return f"TreeNode(key={self.key}, payload={self.payload})"
    
    def __repr__(self):
        return f"TreeNode(key={self.key})"



root = TreeNode(50, "Root")
left = TreeNode(30, "Left", parent=root)
right = TreeNode(70, "Right", parent=root)
    
root.leftChild = left
root.rightChild = right
    
print(f"Root: {root}")
print(f"Root has left child: {root.hasLeftChild()}")
print(f"Root has right child: {root.hasRightChild()}")
print(f"Left is left child: {left.isLeftChild()}")
print(f"Left is leaf: {left.isLeaf()}")
print(f"Root is root: {root.isRoot()}")
    
# 遍历
print("\nIn-order traversal:")
for node in root:
    print(f"  {node}")

Root: TreeNode(key=50, payload=Root)
Root has left child: True
Root has right child: True
Left is left child: True
Left is leaf: True
Root is root: True

In-order traversal:
  TreeNode(key=30, payload=Left)
  TreeNode(key=50, payload=Root)
  TreeNode(key=70, payload=Right)


## 8.7 AVL树
平衡二叉查找树：需要对每个节点跟踪平衡因子（左右子树高度差）。如果每个节点的平衡因子都在-1，0，1之间，则称二叉搜索树为平衡树。

In [1]:
class AVLNode:
    """AVL树节点类"""
    def __init__(self, key, value=None):
        self.key = key          # 节点键值
        self.value = value      # 节点存储的值（可选）
        self.left = None        # 左子节点
        self.right = None       # 右子节点
        self.height = 1         # 节点高度，初始为1
        self.balance = 0        # 平衡因子

class AVLTree:
    """AVL树类"""
    def __init__(self):
        self.root = None  # 树根节点
        self.size = 0     # 树中节点数量
    
    def __len__(self):
        """返回树中节点数量"""
        return self.size
    
    def _height(self, node):
        """获取节点的高度"""
        if node is None:
            return 0
        return node.height
    
    def _update_height(self, node):
        """更新节点的高度和平衡因子"""
        if node is None:
            return
        left_height = self._height(node.left)
        right_height = self._height(node.right)
        node.height = max(left_height, right_height) + 1
        node.balance = left_height - right_height
    
    def _rotate_right(self, y):
        """
        右旋操作
             y          x
            / \        / \
           x   T3  -> T1  y
          / \            / \
         T1 T2          T2 T3
        """
        x = y.left
        T2 = x.right
        
        # 执行旋转
        x.right = y
        y.left = T2
        
        # 更新高度
        self._update_height(y)
        self._update_height(x)
        
        return x  # 返回新的根节点
    
    def _rotate_left(self, x):
        """
        左旋操作
           x              y
          / \            / \
         T1  y    ->    x  T3
            / \        / \
           T2 T3      T1 T2
        """
        y = x.right
        T2 = y.left
        
        # 执行旋转
        y.left = x
        x.right = T2
        
        # 更新高度
        self._update_height(x)
        self._update_height(y)
        
        return y  # 返回新的根节点
    
    def _rebalance(self, node):
        """重新平衡节点，返回平衡后的节点"""
        if node is None:
            return node
        
        # 更新节点高度
        self._update_height(node)
        
        # 左子树比右子树高（左左或左右情况）
        if node.balance > 1:
            # 左左情况 - 直接右旋
            if node.left.balance >= 0:
                return self._rotate_right(node)
            # 左右情况 - 先左旋左子树，再右旋
            else:
                node.left = self._rotate_left(node.left)
                return self._rotate_right(node)
        
        # 右子树比左子树高（右右或右左情况）
        elif node.balance < -1:
            # 右右情况 - 直接左旋
            if node.right.balance <= 0:
                return self._rotate_left(node)
            # 右左情况 - 先右旋右子树，再左旋
            else:
                node.right = self._rotate_right(node.right)
                return self._rotate_left(node)
        
        # 节点平衡，无需旋转
        return node
    
    def insert(self, key, value=None):
        """插入键值对"""
        self.root = self._insert(self.root, key, value)
        self.size += 1
    
    def _insert(self, node, key, value):
        """递归插入辅助函数"""
        # 找到插入位置，创建新节点
        if node is None:
            return AVLNode(key, value)
        
        # 递归插入
        if key < node.key:
            node.left = self._insert(node.left, key, value)
        elif key > node.key:
            node.right = self._insert(node.right, key, value)
        else:
            # 键已存在，更新值
            node.value = value
            self.size -= 1  # 因为键已存在，所以不增加大小
            return node
        
        # 重新平衡节点
        return self._rebalance(node)
    
    def delete(self, key):
        """删除指定键的节点"""
        if self.root is None:
            return False
        
        # 记录删除前的节点数
        old_size = self.size
        
        # 执行删除
        self.root = self._delete(self.root, key)
        
        # 如果节点数减少了，说明删除成功
        return self.size < old_size
    
    def _delete(self, node, key):
        """递归删除辅助函数"""
        if node is None:
            return None
        
        # 查找要删除的节点
        if key < node.key:
            node.left = self._delete(node.left, key)
        elif key > node.key:
            node.right = self._delete(node.right, key)
        else:
            # 找到要删除的节点
            self.size -= 1
            
            # 情况1：节点是叶子节点或只有一个子节点
            if node.left is None:
                return node.right
            elif node.right is None:
                return node.left
            
            # 情况2：节点有两个子节点
            # 找到右子树中的最小节点（或左子树中的最大节点）
            min_larger_node = self._min_node(node.right)
            
            # 复制最小节点的数据到当前节点
            node.key = min_larger_node.key
            node.value = min_larger_node.value
            
            # 删除右子树中的最小节点
            node.right = self._delete(node.right, min_larger_node.key)
        
        # 重新平衡节点
        return self._rebalance(node)
    
    def _min_node(self, node):
        """找到子树中的最小节点"""
        current = node
        while current.left is not None:
            current = current.left
        return current
    
    def search(self, key):
        """查找指定键的值"""
        node = self._search(self.root, key)
        return node.value if node else None
    
    def _search(self, node, key):
        """递归查找辅助函数"""
        if node is None or node.key == key:
            return node
        
        if key < node.key:
            return self._search(node.left, key)
        else:
            return self._search(node.right, key)
    
    def contains(self, key):
        """检查树是否包含指定键"""
        return self.search(key) is not None
    
    def inorder_traversal(self):
        """中序遍历，返回排序后的键列表"""
        result = []
        self._inorder_traversal(self.root, result)
        return result
    
    def _inorder_traversal(self, node, result):
        """递归中序遍历辅助函数"""
        if node is not None:
            self._inorder_traversal(node.left, result)
            result.append(node.key)
            self._inorder_traversal(node.right, result)
    
    def preorder_traversal(self):
        """前序遍历"""
        result = []
        self._preorder_traversal(self.root, result)
        return result
    
    def _preorder_traversal(self, node, result):
        """递归前序遍历辅助函数"""
        if node is not None:
            result.append(node.key)
            self._preorder_traversal(node.left, result)
            self._preorder_traversal(node.right, result)
    
    def postorder_traversal(self):
        """后序遍历"""
        result = []
        self._postorder_traversal(self.root, result)
        return result
    
    def _postorder_traversal(self, node, result):
        """递归后序遍历辅助函数"""
        if node is not None:
            self._postorder_traversal(node.left, result)
            self._postorder_traversal(node.right, result)
            result.append(node.key)
    
    def level_order_traversal(self):
        """层序遍历"""
        if self.root is None:
            return []
        
        result = []
        queue = [self.root]
        
        while queue:
            level_size = len(queue)
            level = []
            
            for _ in range(level_size):
                node = queue.pop(0)
                level.append(node.key)
                
                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
            
            result.append(level)
        
        return result
    
    def get_height(self):
        """获取树的高度"""
        return self._height(self.root)
    
    def is_balanced(self):
        """检查树是否平衡"""
        return self._is_balanced(self.root)
    
    def _is_balanced(self, node):
        """递归检查平衡辅助函数"""
        if node is None:
            return True
        
        left_balanced = self._is_balanced(node.left)
        right_balanced = self._is_balanced(node.right)
        
        if not left_balanced or not right_balanced:
            return False
        
        self._update_height(node)
        return abs(node.balance) <= 1
    
    def print_tree(self):
        """以可视化的方式打印树结构"""
        if self.root is None:
            print("空树")
            return
        
        lines = []
        self._build_tree_string(self.root, "", "", lines)
        print("\n".join(lines))
    
    def _build_tree_string(self, node, prefix, children_prefix, lines):
        """递归构建树的可视化字符串"""
        if node is not None:
            # 添加当前节点
            lines.append(f"{prefix}{node.key}(h:{node.height}, b:{node.balance})")
            
            # 递归添加子节点
            if node.left is not None or node.right is not None:
                if node.left is not None:
                    self._build_tree_string(node.left, 
                                          children_prefix + "├── ", 
                                          children_prefix + "│   ", 
                                          lines)
                else:
                    lines.append(children_prefix + "├── None")
                
                if node.right is not None:
                    self._build_tree_string(node.right, 
                                          children_prefix + "└── ", 
                                          children_prefix + "    ", 
                                          lines)
                else:
                    lines.append(children_prefix + "└── None")

In [2]:
# 创建AVL树
avl = AVLTree()

# 插入元素
keys = [10, 20, 30, 40, 50, 25]
for key in keys:
    avl.insert(key, f"value_{key}")

print(f"树的大小: {len(avl)}")
print(f"树的高度: {avl.get_height()}")
print(f"树是否平衡: {avl.is_balanced()}")

# 中序遍历（有序输出）
print(f"中序遍历: {avl.inorder_traversal()}")

# 查找元素
print(f"查找键30: {avl.search(30)}")
print(f"查找键100: {avl.search(100)}")

# 检查是否包含
print(f"是否包含20: {avl.contains(20)}")
print(f"是否包含99: {avl.contains(99)}")

# 前序遍历
print(f"前序遍历: {avl.preorder_traversal()}")

# 层序遍历
print(f"层序遍历: {avl.level_order_traversal()}")

# 可视化打印树
print("\n树结构:")
avl.print_tree()

# 删除元素
print(f"\n删除键30: {avl.delete(30)}")
print(f"删除后树的大小: {len(avl)}")
print(f"删除后中序遍历: {avl.inorder_traversal()}")
print(f"删除后树是否平衡: {avl.is_balanced()}")

print("\n删除后的树结构:")
avl.print_tree()

树的大小: 6
树的高度: 3
树是否平衡: True
中序遍历: [10, 20, 25, 30, 40, 50]
查找键30: value_30
查找键100: None
是否包含20: True
是否包含99: False
前序遍历: [30, 20, 10, 25, 40, 50]
层序遍历: [[30], [20, 40], [10, 25, 50]]

树结构:
30(h:3, b:0)
├── 20(h:2, b:0)
│   ├── 10(h:1, b:0)
│   └── 25(h:1, b:0)
└── 40(h:2, b:-1)
    ├── None
    └── 50(h:1, b:0)

删除键30: True
删除后树的大小: 4
删除后中序遍历: [10, 20, 25, 40, 50]
删除后树是否平衡: True

删除后的树结构:
40(h:3, b:1)
├── 20(h:2, b:0)
│   ├── 10(h:1, b:0)
│   └── 25(h:1, b:0)
└── 50(h:1, b:0)


本章小结：表达式解析求值的二叉树；实现ADT Map的二叉查找树BST；改进性能，用于ADT Map的平衡二叉查找树AVL；实现了最小堆的完全二叉树：二叉堆，用于优先队列。